<a href="https://colab.research.google.com/github/jog822/CRISPR-GEM/blob/main/Kallisto_sra_downloads_Official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All Together
Just make sure the correct SRRs are added.To save the new TPM set - uncomment the last line.

In [ ]:
import subprocess
import os
import pandas as pd
commands = [
    "wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz",
    "tar -xzf sratoolkit.current-ubuntu64.tar.gz",
    "rm sratoolkit.current-ubuntu64.tar.gz",
    "wget https://github.com/pachterlab/kallisto/releases/download/v0.46.1/kallisto_linux-v0.46.1.tar.gz",
    "tar -xzf kallisto_linux-v0.46.1.tar.gz",
    "rm kallisto_linux-v0.46.1.tar.gz",
    "wget https://ftp.ensembl.org/pub/release-111/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz"
]
# Execute the commands
for command in commands:
    subprocess.run(command, shell=True, check=True)
if not os.path.isfile("/content/transcripts.idx"):
        subprocess.run(['/content/kallisto/kallisto', "index", "-i", "/content/transcripts.idx", "/content/Homo_sapiens.GRCh38.cdna.all.fa.gz"], check=True)


#Make sure only new datasets are downloaded.
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
    print("Google Drive mounted.")
else:
    print("Google Drive is already mounted.")
import pandas as pd
if 'hello' not in globals():
    hello = pd.read_csv('/content/drive/MyDrive/TIGR_/TPM.csv', index_col=0)
    # hello = pd.read_csv('/content/drive/MyDrive/TIGR_/TPM.csv', index_col=0)
    print("DataFrame 'hello' has been created.")
else:
    print("DataFrame 'hello' already exists.")
already=hello.columns.tolist()
##EITHER:
srr_list=['SRR25880533']
##OR
# listher=pd.read_excel('/content/drive/MyDrive/GEM/*.xlsx', sheet_name='Sheet1')
# srr_list=listher['SRR ID'].tolist()
srr_list = [item for item in srr_list if item not in already]



#Make sure the Necessary Directories have been made.
folder_path = '/content/drive/MyDrive/GEM'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at {folder_path}")
else:
    print(f"Folder already exists at {folder_path}")
folder_path = '/content/drive/MyDrive/GEM/DataDownload'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at {folder_path}")
else:
    print(f"Folder already exists at {folder_path}")

#Download and prepare the data
OUTPUT_FOLDER=folder_path
def process_srr(srr, OUTPUT_FOLDER):
  try:
    output_folder = OUTPUT_FOLDER+f"/output_{srr}"
    if os.path.exists(output_folder) and os.listdir(output_folder):
        print(f"Skipping {srr} since output_{srr} exists and is not empty.")
        return
    subprocess.run(["/content/sratoolkit.3.1.1-ubuntu64/bin/prefetch", srr], check=True)
    print(f"{srr} Prefetch Finished")
    subprocess.run(["/content/sratoolkit.3.1.1-ubuntu64/bin/fasterq-dump", srr], check=True)
    print(f"{srr} Fasterqdump Finished")
    if not os.path.isfile("/content/transcripts.idx"):
        subprocess.run(["/content/kallisto/kallisto", "index", "-i", "/content/transcripts.idx", "/content/Homo_sapiens.GRCh38.cdna.all.fa.gz"], check=True)
    print("Transcripts Present")
    if os.path.isfile(f"{srr}_2.fastq"):
        print(f"{srr} is paired-end")
        subprocess.run(["/content/kallisto/kallisto", "quant", "-i", "/content/transcripts.idx", "-o", OUTPUT_FOLDER+f"/output_{srr}", "-b", "100", f"{srr}_1.fastq", f"{srr}_2.fastq"],check=True)

        os.remove(f"{srr}_1.fastq")
        os.remove(f"{srr}_2.fastq")
        os.remove(f"{srr}/{srr}.sra")
    else:
        print(f"{srr} is single-end")
        subprocess.run(["/content/kallisto/kallisto", "quant", "-i", "/content/transcripts.idx", "-o", OUTPUT_FOLDER+f"/output_{srr}", "-b", "100", "-l", "200", "-s", "20", "--single", f"{srr}.fastq"], check=True)
        os.remove(f"{srr}.fastq")
        os.remove(f"{srr}.sra")
    print(f"Processing {srr} finished")
  except Exception as E:
    print(f"Error processing {srr}: {E}")

for srr in srr_list:
  process_srr(srr, OUTPUT_FOLDER)
print("All done ;)")



#Append the new data to the TPM Dataset.
result_df = pd.DataFrame()
for folder in os.listdir(OUTPUT_FOLDER):
    folder_path = os.path.join(OUTPUT_FOLDER, folder)
    if os.path.isdir(folder_path):
        tsv_file_path = os.path.join(folder_path, "abundance.tsv")
        if os.path.exists(tsv_file_path):
            tsv_df = pd.read_csv(tsv_file_path, sep='\t', index_col=0)
            est_counts = tsv_df['tpm'].rename(folder)
            # est_counts = tsv_df['est_counts'].rename(folder)
            result_df = pd.concat([result_df, est_counts], axis=1)
            result_df.columns = result_df.columns.str.replace('output_', '', regex=True)
TPM=pd.concat([hello, result_df], axis=1)
# TPM.to_csv('/content/drive/MyDrive/TIGR_/TPM.csv')
#TPM.to_csv('/content/drive/MyDrive/TIGR_/Big.csv')

Folder already exists at /content/drive/MyDrive/GEM
Folder created at /content/drive/MyDrive/GEM/DataDownload
SRR25880533 Prefetch Finished
SRR25880533 Fasterqdump Finished
Transcripts Present
SRR25880533 is paired-end
Processing SRR25880533 finished
All done ;)


# Individual Blocks
Intended only for troubleshooting. Ignore these for new data downloads

In [ ]:
import subprocess
import os
import pandas as pd
commands = [
    "wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz",
    "tar -xzf sratoolkit.current-ubuntu64.tar.gz",
    "rm sratoolkit.current-ubuntu64.tar.gz",
    "wget https://github.com/pachterlab/kallisto/releases/download/v0.46.1/kallisto_linux-v0.46.1.tar.gz",
    "tar -xzf kallisto_linux-v0.46.1.tar.gz",
    "rm kallisto_linux-v0.46.1.tar.gz",
    "wget https://ftp.ensembl.org/pub/release-111/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz"
]
# Execute the commands
for command in commands:
    subprocess.run(command, shell=True, check=True)
if not os.path.isfile("/content/transcripts.idx"):
        subprocess.run(['/content/kallisto/kallisto', "index", "-i", "/content/transcripts.idx", "/content/Homo_sapiens.GRCh38.cdna.all.fa.gz"], check=True)

In [ ]:
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
    print("Google Drive mounted.")
else:
    print("Google Drive is already mounted.")
import pandas as pd
if 'hello' not in globals():
    hello = pd.read_csv('/content/drive/MyDrive/TIGR_/TPM.csv', index_col=0)
    print("DataFrame 'hello' has been created.")
else:
    print("DataFrame 'hello' already exists.")
already=hello.columns.tolist()
##EITHER:
srr_list=['SRR25880533']
##OR
# listher=pd.read_excel('/content/drive/MyDrive/GEM/*.xlsx', sheet_name='Sheet1')
# srr_list=listher['SRR ID'].tolist()
srr_list = [item for item in srr_list if item not in already]

Google Drive is already mounted.


In [ ]:
OUTPUT_FOLDER=folder_path
def process_srr(srr, OUTPUT_FOLDER):
  try:
    output_folder = OUTPUT_FOLDER+f"/output_{srr}"
    if os.path.exists(output_folder) and os.listdir(output_folder):
        print(f"Skipping {srr} since output_{srr} exists and is not empty.")
        return
    subprocess.run(["/content/sratoolkit.3.1.1-ubuntu64/bin/prefetch", srr], check=True)
    print(f"{srr} Prefetch Finished")
    subprocess.run(["/content/sratoolkit.3.1.1-ubuntu64/bin/fasterq-dump", srr], check=True)
    print(f"{srr} Fasterqdump Finished")
    if not os.path.isfile("/content/transcripts.idx"):
        subprocess.run(["/content/kallisto/kallisto", "index", "-i", "/content/transcripts.idx", "/content/Homo_sapiens.GRCh38.cdna.all.fa.gz"], check=True)
    print("Transcripts Present")
    if os.path.isfile(f"{srr}_2.fastq"):
        print(f"{srr} is paired-end")
        subprocess.run(["/content/kallisto/kallisto", "quant", "-i", "/content/transcripts.idx", "-o", OUTPUT_FOLDER+f"/output_{srr}", "-b", "100", f"{srr}_1.fastq", f"{srr}_2.fastq"],check=True)

        os.remove(f"{srr}_1.fastq")
        os.remove(f"{srr}_2.fastq")
        os.remove(f"{srr}/{srr}.sra")
    else:
        print(f"{srr} is single-end")
        subprocess.run(["/content/kallisto/kallisto", "quant", "-i", "/content/transcripts.idx", "-o", OUTPUT_FOLDER+f"/output_{srr}", "-b", "100", "-l", "200", "-s", "20", "--single", f"{srr}.fastq"], check=True)
        os.remove(f"{srr}.fastq")
        os.remove(f"{srr}.sra")
    print(f"Processing {srr} finished")
  except Exception as E:
    print(f"Error processing {srr}: {E}")

for srr in srr_list:
  process_srr(srr, OUTPUT_FOLDER)
print("All done ;)")

In [ ]:
import os
import pandas as pd
# output_folder = "DataDownload"
result_df = pd.DataFrame()
for folder in os.listdir(OUTPUT_FOLDER):
    folder_path = os.path.join(OUTPUT_FOLDER, folder)
    if os.path.isdir(folder_path):
        tsv_file_path = os.path.join(folder_path, "abundance.tsv")
        if os.path.exists(tsv_file_path):
            tsv_df = pd.read_csv(tsv_file_path, sep='\t', index_col=0)
            est_counts = tsv_df['est_counts'].rename(folder)
            est_counts = tsv_df['tpm'].rename(folder)
            result_df = pd.concat([result_df, est_counts], axis=1)
            result_df.columns = result_df.columns.str.replace('output_', '', regex=True)
TPM=pd.concat([hello, result_df], axis=1)
# TPM.to_csv('/content/drive/MyDrive/TIGR_/TPM.csv')

,Unnamed: 0,SRR13018695,SRR13018696,SRR13018697,SRR13018698,SRR13018699,SRR13018700,SRR13018701,SRR13018702,SRR13018703,...,SRR6355994,SRR6355995,SRR6355996,SRR6355997,SRR6355998,SRR6355999,SRR6356000,SRR6356001,SRR6356002,output_SRR25880533
0,ENST00000415118.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,ENST00000448914.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,ENST00000434970.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,ENST00000631435.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,ENST00000632684.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
